In [6]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

In [7]:
#qlib交易策略
path='output/patchtst/pred.pkl'
f=open(path,'rb')
data=pickle.load(f)
df=pd.DataFrame(data)
df.reset_index(inplace=True)
df.rename(columns={'kdcode':'instrument','dt':'datetime'},inplace=True)
df.rename(columns={'Company':'instrument','Date':'datetime'},inplace=True)
df

,datetime,instrument,score,label
0,2023-01-03,SH600000,0.003582,0.005472
1,2023-01-03,SH600009,0.004154,0.033847
2,2023-01-03,SH600010,0.003925,-0.005155
3,2023-01-03,SH600011,0.003782,0.011583
4,2023-01-03,SH600015,0.003276,0.000000
...,...,...,...,...
75339,2023-12-29,SZ300919,0.004110,0.001863
75340,2023-12-29,SZ300957,0.003654,-0.000919
75341,2023-12-29,SZ300979,0.003546,-0.021997
75342,2023-12-29,SZ300999,0.003719,0.000000


In [ ]:
x=df
trade_date_unique=df['datetime'].unique()
df_return=pd.DataFrame()
#b={'datetime':trade_date_unique[0],'daily_return':0}
#df_return=df_return.append(b,ignore_index=True)
for date in tqdm(trade_date_unique):
    x_day=x[x['datetime']==date]
    # TODO: topk / top m, 每天只能交易其中 n 支股票, n / m = 0.1
    r_day=x_day.nlargest(9,columns='score').label.mean()
    # r_day=r_day-1
    # r_day-=x_day.nsmallest(9,columns='score').label.mean() 做空
    b={'datetime':date,'daily_return':r_day}
    df_return=df_return._append(b,ignore_index=True)

100%|██████████| 242/242 [00:01<00:00, 216.40it/s]


In [30]:
portfolio_df_performance = df_return.set_index(['datetime'])
portfolio_df_performance

,daily_return
datetime,
2023-01-03,0.002955
2023-01-04,0.005367
2023-01-05,0.001652
2023-01-06,0.000120
2023-01-09,0.003204
...,...
2023-12-25,-0.013943
2023-12-26,0.000206
2023-12-27,0.015918


In [1]:
ARR = (1 + portfolio_df_performance).cumprod()
ARR

NameError: name 'portfolio_df_performance' is not defined

In [32]:
alpha_df_performance = pd.DataFrame()
alpha_df_performance['portfolio_daily_return'] = portfolio_df_performance['daily_return']
alpha_df_performance['portfolio_net_value'] = (alpha_df_performance['portfolio_daily_return'] + 1).cumprod()

net_value_columns = ['portfolio_net_value']

alpha_statistics_df = pd.DataFrame(index=alpha_df_performance[net_value_columns].columns,
                                    columns=["年化收益", "年化波动率", "最大回撤率", "夏普率", "Calmar", "IR"])

# alpha_df_performance.set_index("dt", inplace=True)
alpha_df_performance.index = pd.to_datetime(alpha_df_performance.index)
monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
monthly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    monthly_statistics_df])
monthly_statistics_df = monthly_statistics_df.pct_change()
monthly_statistics_df = monthly_statistics_df.dropna()
monthly_statistics_df.index = monthly_statistics_df.index.date

yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
yearly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    yearly_statistics_df])
yearly_statistics_df = yearly_statistics_df.pct_change()
yearly_statistics_df = yearly_statistics_df.dropna()
yearly_statistics_df.index = yearly_statistics_df.index.date

alpha_statistics_df.loc[:, "年化收益"] = np.mean(
    (alpha_df_performance[net_value_columns].tail(1)) ** (252 / len(alpha_df_performance)) - 1)
alpha_statistics_df.loc[:, "年化波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252)
alpha_statistics_df.loc[:, "累积收益"] = np.mean(alpha_df_performance[net_value_columns].tail(1) - 1)
alpha_statistics_df.loc[:, "累积波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)
alpha_statistics_df.loc[:, "最大回撤率"] = np.min(
    (alpha_df_performance[net_value_columns] - alpha_df_performance[net_value_columns].cummax()) /
    alpha_df_performance[net_value_columns].cummax())
alpha_statistics_df.loc[:, "夏普率"] = alpha_statistics_df["年化收益"] / alpha_statistics_df["年化波动率"]
alpha_statistics_df.loc[:, "Calmar"] = alpha_statistics_df["年化收益"] / abs(alpha_statistics_df["最大回撤率"])
alpha_statistics_df.loc[:, "IR"] = np.mean(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252) / np.std(alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)

/home/huyifan/miniconda3/envs/ts/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/tmp/ipykernel_2489137/2453866015.py:28: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  alpha_statistics_df.loc[:, "年化收益"] = np.mean(
/tmp/ipykernel_2489137/2453866015.py:30: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(

In [33]:
alpha_statistics_df

,年化收益,年化波动率,最大回撤率,夏普率,Calmar,IR,累积收益,累积波动率
portfolio_net_value,0.001817,0.195393,-0.1379,0.0093,0.013177,0.090979,0.001745,0.012309


ASR = ARR / AVol;   CR = ARR / abs(MDD)
                        ARR    AVol    MDD    ASR     CR    IR
hs300  lstm            -0.214  0.157  -0.275 -1.361 -0.779 -1.492
       lstm tra        -0.216  0.164  -0.294 -1.314 -0.735 -1.461
       gru             -0.229  0.156  -0.290 -1.469 -0.790 -1.631
       transformer     -0.240  0.156  -0.281 -1.543 -0.855 -1.695
       transformer tra -0.074  0.169  -0.222 -0.436 -0.332 -0.409
       mamba           -0.241  0.155  -0.290 -1.552 -0.830 -1.740
       tsformer        -0.337  0.180  -0.369 -1.871 -0.914 -2.217
       patchTST        -0.224  0.158  -0.279 -1.415 -0.803 -1.563
       Crossformer     -0.071  0.162  -0.179 -0.437 -0.395 -0.383
       iTransformer    -0.115  0.145  -0.190 -0.793 -0.607 -0.775
       TimeMixer       -0.156  0.159  -0.232 -0.983 -0.674 -1.028
      
zz500  lstm            -0.008  0.159  -0.172 -0.047 -0.044 -0.128
       lstm tra         0.016  0.162  -0.192  0.101  0.086  0.014
       gru             -0.004  0.159  -0.193 -0.028 -0.023 -0.118
       transformer      0.154  0.156  -0.135  0.986  1.143  0.867
       transformer tra  0.125  0.162  -0.145  0.776  0.866  0.657
       mamba            0.056  0.157  -0.180  0.357  0.312  0.297
                        0.086  0.156  -0.143  0.551  0.601  0.456
       tsformer         0.285  0.203  -0.196  1.405  1.453  1.317
       patchTST         0.118  0.152  -0.127  0.776  0.923  0.735
       Crossformer     -0.039  0.163  -0.217 -0.238 -0.179 -0.350
       iTransformer     0.214  0.168  -0.164  1.276  1.309  1.173
       TimeMixer        0.078  0.153  -0.114  0.511  0.685  0.385
      
sp500  lstm            0.142  0.162  -0.178  0.877  0.798  0.929
       lstm tra        0.191  0.161  -0.150  1.186  1.273  1.115
       gru             0.124  0.169  -0.139  0.734  0.829  1.023
       transformer     0.135  0.159  -0.140  0.852  0.968  0.908
       transformer tra 0.184  0.166  -0.158  1.114  1.172  1.106
       mamba           0.131  0.170  -0.156  0.771  0.837  0.832
                       0.187  0.173  -0.156  1.081  1.199  1.188
       tsformer        0.326  0.169  -0.142  1.927  2.298  1.842
       patchTST        0.146  0.167  -0.140  0.877  1.042  0.949
       Crossformer     0.284  0.159  -0.114  1.786  2.491  1.646
       iTransformer    0.159  0.170  -0.139  0.941  1.150  0.955
       TimeMixer       0.254  0.162  -0.131  1.568  1.938  1.448
      
nas100 lstm            0.247  0.176  -0.128  1.403  1.930  1.386
       lstm tra        0.201  0.192  -0.183  1.047  1.098  1.032
       gru             0.225  0.188  -0.165  1.197  1.364  1.160
       transformer     0.268  0.175  -0.131  1.531  2.046  1.441
       transformer tra 0.267  0.181  -0.144  1.475  1.854  1.427
       mamba           0.230  0.170  -0.143  1.353  1.610  1.304
       tsformer        0.347  0.186  -0.104  1.859  3.340  1.738
       patchTST        0.239  0.185  -0.138  1.296  1.735  1.233
       Crossformer     0.363  0.181  -0.167  2.010  2.180  1.860
       iTransformer    0.188  0.196  -0.202  0.963  0.932  0.937
       TimeMixer       0.264  0.188  -0.131  1.401  2.012  1.346
       